# Problem 0. Your Background and Interests (5 points)

### a) Deeper and better understandment of Machine Learning algorithims and techniques.


### b) Big Data

### c) I'm an industrial engineering major so the curriculum has a heavy load of statistics classes

### d) I have used Python and Scala for machine learning
<br>

# Problem 1. Nearest Neighbors: Training Set Size and Noise (40 points) 
---

#### Import Libraries

In [1]:
#Scipy & sklearn for .mat file read, distance and KFold
import scipy.io as sio; from scipy.spatial import distance; from sklearn.model_selection import KFold
#Numpy and Pandas for Data Wrangling
import numpy as np; import pandas as pd
# Plotly for Visualization
import plotly.graph_objs as go ;from plotly import tools ;from plotly.offline import iplot,init_notebook_mode
import matplotlib.pyplot as plt
init_notebook_mode(connected=True)
#Python Useful Libraries
from collections import Counter ;import random ;from itertools import product

# 1a: 

### Read MNIST Data From File

In [2]:
mat_contents = sio.loadmat('mnistData.mat')
mat_contents['mnist'].dtype

dtype([('train_images', 'O'), ('test_images', 'O'), ('train_labels', 'O'), ('test_labels', 'O')])

### Reshape images to 1000 x 784 matrix

In [3]:
n=1000
Xtrain=np.reshape(np.transpose(mat_contents['mnist']['train_images'][0,0][:,:,0:n]),(n,28*28))
Xtest=np.reshape(np.transpose(mat_contents['mnist']['test_images'][0,0][:,:,0:n]),(n,28*28))
Ytrain=mat_contents['mnist']['train_labels'][0,0][0:n].flatten()
Ytest=mat_contents['mnist']['test_labels'][0,0][0:n].flatten()

### 1KNN Algorithim 

In [4]:
def KNN1(Xtrain,Ytrain,Xtest,Ytest):
    
    ypred=np.zeros(len(Ytest)) #Fill Array with zeroes

    for i in range(len(Xtest)): #Iterate over Xtest data points
        distances=[]
        
        for datapoint in Xtrain:
            distances.append(distance.sqeuclidean(Xtest[i],datapoint)) #Calculate the squared Euclidean Distance
            
        min_dist = np.argmin(distances) #Take the index of the point with the smallest distance
        ypred[i]=Ytrain[min_dist] #Get that datapoint label
        
    return np.mean(ypred!=Ytest) #Return the Test Error

print "Test Error: {0:.2f}".format(KNN1(Xtrain,Ytrain,Xtest,Ytest) * 100)

Test Error: 17.20


### KFold Cross Validation Algorithim

In [5]:
def k_fold_cross_validation(X,Y,k):

    kf = KFold(n_splits=k)  #Get the indices of k splits
    errors=[]
    for train_index, test_index in kf.split(X): #Iterate over the k splits of the Xtrain DataSet
        
        x_train, x_test = X[train_index], X[test_index] #Get the xtrain and xtest of that split
        y_train, y_test = Y[train_index], Y[test_index] #Get the ytrain and ytest if that split
        errors.append(KNN1(x_train,y_train,x_test,y_test)) #Append the Test Error to list of K errors
        
    return np.mean(errors) #Return the mean of the k samples validation error

### N-Fold Cross Validation Error Plot

In [6]:
sizes=[3, 10, 50, 100, 1000] #N Sizes for Folds
errors=[ k_fold_cross_validation(Xtrain,Ytrain,k) for k in sizes] #Get the error for each NFold Cross Validation

trace = go.Scatter( x = sizes, y = errors) #Line Scatter Plot

# Customize Layout
layout = go.Layout( title="N-Fold Cross Validation Error",
                    xaxis=dict( title="N", type= "category"),
                    yaxis=dict(title="Error Rate"))

fig=dict(data=[trace],layout=layout)
iplot(fig)

___

# 1b:

### Read Iris Data From File

In [7]:
iris=pd.read_csv("iris.csv",header=None,names=['X1','X2','Target'])

iris.head(5)

,X1,X2,Target
0,2.3330,0.3509,1
1,2.9453,0.2621,1
2,3.0889,0.1536,1
3,3.0566,0.1285,1
4,3.0205,0.2571,1


### KNN Algorithim

In [8]:
def KNN(Xtrain, Ytrain, Xtest, k):
    
    ypred=np.zeros(len(Xtest))
    
    for i in range(len(Xtest)):
        distances=[]
        for datapoint in Xtrain:
            distances.append(distance.sqeuclidean(Xtest[i],datapoint))
            
        sorted_distances = np.argsort(distances)[:k] #Take the K indices of the smalles distances
        
        ypred[i]= Counter([Ytrain[j] for j in sorted_distances]).most_common()[0][0] #Choose the majority labael
   
    return ypred

### 1KNN Decision Boundary Plot On Original Iris Data

In [9]:
xtrain=iris[['X1','X2']].as_matrix() # Transform features to matrix
ytrain=iris['Target'].as_matrix() 

x_min, x_max = xtrain[:, 0].min() - 1, xtrain[:, 0].max() + 1 #Get Mins and Maxs of Features
y_min, y_max = xtrain[:, 1].min() - 1, xtrain[:, 1].max() + 1 #Get Mins and Maxs of Features

xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1)) #Construct meshgrid
y_ = np.arange(y_min, y_max, 0.1)

Z = KNN(xtrain,ytrain,np.c_[xx.ravel(), yy.ravel()],1).reshape(xx.shape) #Get the predicted decision labels

#Draw Decision Boundaries and Contours
trace1 = go.Contour(x=xx[0], y=y_, z=Z, colorscale=[[0, '#FF0000'], [0.5, '#00FF00'],[1, 'blue']],
                    opacity=0.5, showscale=False )

# Draw the data points
trace2 = go.Scatter(x=xtrain[:,0], y=xtrain[:,1], showlegend=False, mode='markers',
                        marker=dict(color=ytrain, line=dict(color='black', width=1)))
# Customize layout
layout = go.Layout(title="N-Fold Cross Validation Error", xaxis=dict(showgrid=False, zeroline=False, range=[2,5]),
                   yaxis=dict(showgrid=False, zeroline=False, range=[0,3]), height=600, width=650)

fig=dict(data=[trace1,trace2],layout=layout)
iplot(fig)

### Decision Boundary For m Random Samples using 3NN classifier

In [10]:
samples_sizes=[10, 20, 30, 50] # M random samples

titles=['{} sample 3NN Decision Boundary'.format(size) for size in samples_sizes] #Plots title

fig = tools.make_subplots(rows=2, cols=2,print_grid=False,subplot_titles=titles) #Make a figure for 4 plots

errors=[]

for idx, sample_size in zip(product([0, 1], [0, 1]), samples_sizes): #Iterate of sample sizes
    
    sample=iris.sample(n=sample_size) #Draw n random sample from dataset
    
    xtrain=sample[['X1','X2']].as_matrix() #Transform features to matrix
    ytrain=sample['Target'].as_matrix()
    
    flip=[[2,3],[1,3],[1,2]] # Array of 2-labels combinations
    
    ytrain=[random.choice(flip[i-1]) for i in ytrain] #Flip the label by choosing randomly the other two labels
    
    x_min, x_max = xtrain[:, 0].min() - 1, xtrain[:, 0].max() + 1 #Get Mins and Maxs of Features
    y_min, y_max = xtrain[:, 1].min() - 1, xtrain[:, 1].max() + 1 #Get Mins and Maxs of Features
    y_ = np.arange(y_min, y_max, 0.1)
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1)) #Construct meshgrid

    Z = KNN(xtrain,ytrain,np.c_[xx.ravel(), yy.ravel()],3).reshape(xx.shape) #Get the predicted decision labels
    
    errors.append(np.mean(KNN(xtrain,ytrain,xtrain,3) != ytrain)) #Get the Training Error
    
    #Draw Decision Boundaries and Contours
    trace1 = go.Contour(x=xx[0], y=y_, z=Z, colorscale=[[0, '#FF0000'], [0.5, '#00FF00'],[1, 'blue']],
                        opacity=0.5, showscale=False )
    
    # Draw the data points
    trace2 = go.Scatter(x=xtrain[:,0], y=xtrain[:,1], 
                        showlegend=False, mode='markers',
                        marker=dict(color=ytrain, line=dict(color='black', width=1)))
    
    #Append plot to figure
    fig.append_trace(trace1, idx[0]+1, idx[1]+1)
    fig.append_trace(trace2, idx[0]+1, idx[1]+1)
    
fig['layout'].update(height=900, hovermode='closest') #Adjust size

#Customize Layout and Add Train Error
for error,i in zip(errors,map(str, range(1,5))):
    x = 'xaxis' + i
    y = 'yaxis' + i
    fig['layout'][x].update(title="Train error = {0:.2f}".format(error),showgrid=False, zeroline=False,)
    fig['layout'][y].update(showgrid=False, zeroline=False,)
    
iplot(fig)

# Problem 2. Nearest Neighbors (15 points)
---

In [11]:
df = pd.DataFrame({'x1' : [3, 17, -4, 4, 8, 19], 'x2' : [10,-17,9,0,-1,3], 
                   'x3' : [2,9,-2,2,6,23], 'x4' : [11,-1,-1,-5,-12,14],
                   'y'  : ['Red', 'Blue', 'Red', 'Blue', 'Blue','Red']})
df

,x1,x2,x3,x4,y
0,3,10,2,11,Red
1,17,-17,9,-1,Blue
2,-4,9,-2,-1,Red
3,4,0,2,-5,Blue
4,8,-1,6,-12,Blue
5,19,3,23,14,Red


## 2a

In [12]:
xtest = [0,0,0,0] 
Xtrain = df.ix[:,:-1].as_matrix() #Get Xtrain Set
Ytrain = df['y'].as_matrix() #Get Ytrain set

distances=[]

#Get the Euclidean distances from all five data points, append them to distances list
for datapoint in Xtrain: distances.append(np.sqrt(np.sum([(x-y)**2 for x,y in zip(xtest,datapoint)]))) 

print "Euclidean Distances to each training example {}\n".format([round(x,2) for x in distances])  

pred_1NN = Ytrain[np.argmin(distances)] #Get the prediction by taking the smallest distance

#Get the prediction by taking the majority vote for the kth smallest distances
pred_3NN = Counter([Ytrain[j] for j in np.argsort(distances)[:3]]).most_common()[0][0]

print  "1-NN prediction : {}\n3-NN prediction: {}".format(pred_1NN,pred_3NN) 

Euclidean Distances to each training example [15.3, 25.69, 10.1, 6.71, 15.65, 33.09]

1-NN prediction : Blue
3-NN prediction: Red


## 2b

In [13]:
distances=[]

#Get the Manhattan distances from all five data points, append them to distances list
for datapoint in Xtrain: distances.append(np.sum([x-y for x,y in zip(xtest,datapoint)]))

print "Manhattan Distances to each training example {}\n".format([round(x,2) for x in distances])    

pred_1NN = Ytrain[np.argmin(distances)] #Get the prediction by taking the smallest distance

#Get the prediction by taking the majority vote for the kth smallest distances
pred_3NN = Counter([Ytrain[j] for j in np.argsort(distances)[:3]]).most_common()[0][0]

print "1-NN prediction : {}\n3-NN prediction: {}".format(pred_1NN,pred_3NN)

Manhattan Distances to each training example [-26.0, -8.0, -2.0, -1.0, -1.0, -59.0]

1-NN prediction : Red
3-NN prediction: Red


# 2c

In [14]:
folds_indices = [[1,2,3,4,5],[0,2,3,4,5],[0,1,3,4,5],[0,1,2,4,5],[0,1,2,3,5],[0,1,2,3,4]]

predictions=[]
for i,fold in enumerate(folds_indices):
    distances=[]
    for datapoint in Xtrain[fold]: distances.append(np.sum([x-y for x,y in zip(Xtrain[i],datapoint)]))
    predictions.append(Ytrain[np.argmin(distances)]!=Ytrain[i])
    
print "Leave-One-Out-Cross-Validation (LOOCV) error is {}\n\n\
The examples that contributed to the error are:\n\nThe First Example: {}\nThe Third Example: {}\n\
".format(round(np.mean(predictions),2),Xtrain[predictions][0],Xtrain[predictions][1])

Leave-One-Out-Cross-Validation (LOOCV) error is 0.33

The examples that contributed to the error are:

The First Example: [ 3 10  2 11]
The Third Example: [-4  9 -2 -1]



# Problem 3. Decision Trees (20 points)
---

In [15]:
df = pd.DataFrame({'x1' : [1,-1,-1,1,-1,-1,1,-1,-1,-1], 'x2' : [1,-1,1,-1,-1,-1,-1,1,-1,1], 
                   'x3' : [25,10,7,12,9,6,22,8,7,9],
                   'y'  : [-1,-1,-1,-1,1,-1,-1,1,-1,1]})
df

,x1,x2,x3,y
0,1,1,25,-1
1,-1,-1,10,-1
2,-1,1,7,-1
3,1,-1,12,-1
4,-1,-1,9,1
5,-1,-1,6,-1
6,1,-1,22,-1
7,-1,1,8,1
8,-1,-1,7,-1
9,-1,1,9,1


# 3a

### Hot Encode feature

In [16]:
for point in range(df['x3'].min()+1,df['x3'].max()):
    df['equal to {}'.format(point)]= [1 if x==True else 0 for x in df['x3']==point]

for feature in ['x1','x2']:
    df['{} == 1'.format(feature)] = [1 if x==True else 0 for x in df[feature]==1]

df.drop(['x1','x2','x3'],1,inplace=True)
df

,y,equal to 7,equal to 8,equal to 9,equal to 10,equal to 11,equal to 12,equal to 13,equal to 14,equal to 15,...,equal to 17,equal to 18,equal to 19,equal to 20,equal to 21,equal to 22,equal to 23,equal to 24,x1 == 1,x2 == 1
0,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,-1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,-1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
7,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,-1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [23]:
def num_mistakes(labels_in_node):

    if len(labels_in_node) == 0: return 0
    
    num_of_positive = (labels_in_node == +1).sum()
    num_of_negative = (labels_in_node == -1).sum()
                
    return num_of_negative if num_of_positive > num_of_negative else num_of_positive

def best_splitting_feature(data, features, target):
    
    best_feature = None  
    best_error = 10      
    
    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        left_split = data[data[feature] == 0]    
        right_split =  data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left and right split.
        left_mistakes = num_mistakes(left_split[target])            
        right_mistakes = num_mistakes(right_split[target])
            
        # Compute the classification error of this split.
        error = (left_mistakes + right_mistakes) / num_data_points
        
        #Keep track of the best feature witht the lowest error
        if error < best_error:
            best_feature = feature            
            best_error = error              
    
    return (best_feature,best_error)

def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None, 'left' : None, 'right' : None, 'is_leaf':  True  }  
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    leaf['prediction'] = "{} |  {}".format(num_minus_ones,num_ones)
              
    return leaf

def decision_tree_create(data, features, target, measure_type):
    remaining_features = features[:] 
    
    target_values = data[target]
    
    # If not mistakes at current node, make current node a leaf node
    if num_mistakes(target_values) == 0:      
        return create_leaf(target_values)
    
    # If there are no remaining features to consider, make current node a leaf node
    if remaining_features == []:   
        return create_leaf(target_values)    
    
    if measure_type == 'Classification Error':
        splitting_feature, v = best_splitting_feature(data, features, target)
    else:
        splitting_feature, v = best_splitting_feature_entropy(data,features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    remaining_features.remove(splitting_feature)
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        return create_leaf(right_split[target])

    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, measure_type)        
    right_tree = decision_tree_create(right_split, remaining_features, target, measure_type) 
    
    #"Count the target values"
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    measure= "Classification Error" if measure_type=='Classification Error' else "Information Gain"
    return {'is_leaf'          : False, 
            'prediction'       : "{} | {} \t{}:{}".format(num_minus_ones,num_ones,measure,round(v,2)),
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}


def print_stump(tree):
    split_name = tree['splitting_feature'] 
    branch=[]
    if split_name is None:
        branch.append('leaf, ' + str(tree['left']['prediction']))
        return branch
    branch.append('                  %s' % '-1|+1 = '+tree['prediction'])
    branch.append('         |---------------|----------------|')
    branch.append('         |                                |')
    branch.append('         |                                |')
    branch.append('         |                                |')
    branch.append('  [Not {0} ]                 [{0} ]        '.format(split_name))
    branch.append('         |                                |')
    branch.append('         |                                |')
    branch.append('         |                                |')
    branch.append('  %s                         %s\n' \
        % (('leaf, ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else ' '),
           ('leaf, ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else ' ')))
    return branch

In [24]:
features=df.columns[1:].tolist()
target='y'
tree=decision_tree_create(df,features,target,'Classification Error')

In [25]:
for line in print_stump(tree)[:-1]:
    print "                  {}".format(line)
print"                                   {}".format(print_stump(tree)[-1])
for line in print_stump(tree['left']):
    print "{}".format(line)

                                    -1|+1 = 7 | 3 	Classification Error:0.1
                           |---------------|----------------|
                           |                                |
                           |                                |
                           |                                |
                    [Not equal to 9 ]                 [equal to 9 ]        
                           |                                |
                           |                                |
                           |                                |
                                                               leaf, 0 |  2

                  -1|+1 = 7 | 1 	Classification Error:0.0
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [Not equal to 8 ]                 [equal to 8 ]        
         |                                |
 

# 3b

In [20]:
def calculate_entropy(labels_in_node):

    num_of_positive = (labels_in_node == +1).sum()
    num_of_negative = (labels_in_node == -1).sum()
    
    if len(labels_in_node) == 0 or num_of_positive==0 or num_of_negative==0: return 0
    
    total = float(len(labels_in_node))
    
    positives_entropy = -num_of_positive/total * np.log2(num_of_positive/total)
    negatives_entropy = -num_of_negative/total * np.log2(num_of_negative/total)
    
    return positives_entropy + negatives_entropy

In [21]:
def best_splitting_feature_entropy(data, features, target):
    
    best_feature = None  
    max_gain = -1      
    node_entropy = calculate_entropy(data[target])
    # Convert to float to make sure error gets computed correctly.
    total = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        left_split = data[data[feature] == 0]    
        right_split =  data[data[feature] == 1]
            
        # Calculate the entropy in the left and right split.
        left_entropy = calculate_entropy(left_split[target])            
        right_entropy = calculate_entropy(right_split[target])
            
        # Compute the information gain of this split.
        gain = node_entropy -(len(left_split)/total)*left_entropy - (len(right_split)/total)*right_entropy 
        
        #Keep track of the best feature witht the lowest error
        if gain > max_gain:
            best_feature = feature            
            max_gain = gain              
    
    return (best_feature,max_gain)

In [26]:
features=df.columns[1:].tolist()
target='y'
tree=decision_tree_create(df,features,target,'Entropy')
for line in print_stump(tree)[:-1]:
    print "\t\t{}\n".format(line)
print"                        {}".format(print_stump(tree)[-1])
for line in print_stump(tree['left']):
    print "{}".format(line)

		                  -1|+1 = 7 | 3 	Information Gain:0.45

		         |---------------|----------------|

		         |                                |

		         |                                |

		         |                                |

		  [Not equal to 9 ]                 [equal to 9 ]        

		         |                                |

		         |                                |

		         |                                |

                                                    leaf, 0 |  2

                  -1|+1 = 7 | 1 	Information Gain:0.54
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [Not equal to 8 ]                 [equal to 8 ]        
         |                                |
         |                                |
         |                                |
  leaf, 7 |  0                         leaf, 0 |  1



# 3c

### Same optimal trees as drawn above. The tree height is 2
<br>

# 3d

In the classification error, the information gain after the first split is exactly 0, since average classification error of the two child nodes(8/10 * 7/8 + 0 = 0.333) is exactly the same as the classification error of the parent node (7/10 = 0.333).<br>
In this case, splitting the initial training set wouldn't yield any improvement in terms of our classification error criterion.<br><br>
In contrast to the average classification error, the average child node entropy is not equal to the entropy of the parent node. Thus, the splitting rule would continue until the child nodes are pure<br><br>

# Problem 4. Maximum Likelihood Estimation (20 points)
---

# 4a

The likelihood function is

$$ \mathcal{L}(a,b)= {\displaystyle \prod_{i=1}^{n} f(x_i;a,b)} 
= \displaystyle \prod_{i=1}^{n} \frac{1}{(b-a)} 
= \frac{1}{(b-a)^n} \\
$$ 

Taking the log-likelihood function

$$ \log\mathcal{L}(a,b) = \log{\displaystyle \prod_{i=1}^{n} f(x_i;a,b)}
= \displaystyle \log\prod_{i=1}^{n} \frac{1}{(b-a)}
= \log{\big((b-a)^{-n}\big)} = -n \cdot \log{(b-a)} \\
$$

To find $ \hat{a}_{MLE} $ and $ \hat{b}_{MLE} $, we take the log-likelihood function derivatives with respect to *a* and *b* to zero:

$$ \frac{\partial}{\partial a} \log\mathcal{L}(a,b) = \frac{n}{(b-a)} = 0\\
\frac{\partial}{\partial b} \log \mathcal{L}(a,b) = -\frac{n}{(b-a)} = 0
 $$

Since the derivative with respect to * a * is monotonically increasing, we need the biggest *a*  

$$ \hat{a}_{MLE}=\min(X_1 , ... , X_n) \\$$ 

Since the derivative with respect to * b * is monotonically decreasing, we need the smallest *b*  

$$ \hat{b}_{MLE}=\max(X_1 , ... , X_n) \\$$



# 4b



The likelihood function is

$$ \mathcal{L}(\lambda  ) = \prod_{i=1}^{n}   \frac{\lambda^{x_{i}}e^{-\lambda}}{x_{i}!}   = e^{-\lambda n}      \frac{\lambda^{\sum_{i=1}^{n}x_{i}}}{\prod_{i=1}^{n}x_{i}} \\
$$ 

Taking the log-likelihood function

$$ \log\mathcal{L}(\lambda) = - \lambda n + \sum_{i=1}^{n} xi . \log(\lambda) - \log( \prod_{i=1}^{n}xi) \\
$$

To find $ \hat{\lambda}_{MLE} $ , we take the log-likelihood function derivative in respect to lambda to zero 

$$ \frac{\partial}{\partial \lambda} \log \mathcal{L}(\lambda) = -n + \sum_{i=1}^{n}xi .\frac{1}{\lambda} = 0 \\
 $$

Solving for lambda, we get the expression of the mean

$$ \hat{\lambda}_{MLE} = \frac{\sum_{i=1}^{n}x_{i}}{n} = \overline{X} \\$$ 



